# A. Test some stuff
test some stuff


In [1]:
import sys
!{sys.executable} -m pip install stixorm
from stixorm.module.typedb import TypeDBSink, TypeDBSource
from stixorm.module.authorise import import_type_factory
from stixorm.module.definitions.stix21 import (
    Identity, EmailAddress, UserAccount, Relationship, Bundle, Incident, URL, EmailMessage, ObservedData
)
from stixorm.module.definitions.os_threat import (
    EventCoreExt, Event, SocialMediaContact, ContactNumber, IncidentCoreExt, TaskCoreExt,
    Task, SightingEvidence, Sequence, SequenceExt, AnecdoteExt, Anecdote,
    SightingAnecdote, SightingAlert, SightingContext, SightingExclusion,
    SightingEnrichment, SightingHunt, SightingFramework, SightingExternal
)
from stixorm.module.authorise import import_type_factory
from stixorm.module.typedb_lib.instructions import ResultStatus, Result
from stixorm.module.parsing import parse_objects
import_type = import_type_factory.get_all_imports()
import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

### A.2 Relative Path Imports
https://stackoverflow.com/questions/76162459/jupyter-notebook-importerror-attempted-relative-import-with-no-known-parent-pac/77528726#77528726

In [2]:
import sys
sys.path.append('../')
import os

### A.3 Relative Import of Blocks

In [3]:
import json
from Block_Families.StixORM.SCO.Email_Message.make_email_msg import main as make_email_msg
from Block_Families.StixORM.SRO.Relationship.make_sro import main as make_sro
from Utilities.local_make_sro import invoke_sro_block, invoke_sighting_block
from Utilities.local_make_sdo import (
    invoke_make_observed_data_block, invoke_make_indicator_block, invoke_make_event_block, invoke_make_sequence_block,
    invoke_make_task_block, invoke_make_incident_block
)
from Utilities.local_make_sco import (
    invoke_make_email_addr_block, invoke_make_url_block, invoke_make_e_msg_block,
)
from Utilities.local_make_general import invoke_create_incident_context, invoke_save_incident_context_block, invoke_get_from_company_block, invoke_get_from_user_block, invoke_move_unattached_to_other_block
from conv import conv

context_base = "../Orchestration/Context_Mem/"
path_base = "../Block_Families/Objects/"
results_base = "../Orchestration/Results/"
sro_data = "SRO/Relationship/sro_derived.json"
alert_data = {
    "from": "evil@northkorea.nk",
    "to": "naive@example.com",
    "reporter": "Naive Smith",
    "url": "https://www.northkorea.nk/we/are/mad/"
}
alert_details = {
    "alert_name": "Suspcious Email",
    "alert_source": "user-report",
    "alert_log": "I have found a suspicious email"
}

TR_Context_Memory_Path = "./Context_Mem/Type_Refinery_Context.json"

## C. Collect the Elements of the Observation into an Observed-Data Object

Tbis then represents the potential phishing email as a group of elements

In [4]:

obs_path ="SDO/Observed_Data/observation-try.json"
results_path ="step1/observation-alert.json"
# 3. Invoke the Make Observed Data Block
obs_1 = invoke_make_observed_data_block(obs_path, results_path, [])
# 4. Add the record to the in-session bundles and lists
context_type = {
    "context_type": "unattached"
}
obs_results_context_path = results_base + "step1/observation-try_context.json"
# result = invoke_save_incident_context_block(results_base + results_path, obs_results_context_path, context_type)
# print(f" result->{result}")

{
    "type": "observed-data",
    "spec_version": "2.1",
    "id": "observed-data--de3ad24a-d979-42b1-8a4f-77f9d4efb393",
    "created": "2024-09-01T02:17:05.410Z",
    "modified": "2024-09-01T02:17:05.410Z",
    "first_observed": "2020-10-19T01:01:01Z",
    "last_observed": "2020-10-19T01:01:01Z",
    "number_observed": 1,
    "object_refs": [
        "email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf212",
        "user-account--597ad4d4-35ba-585d-8f6d-134a75032f9b",
        "email-addr--4722424c-7012-56b0-84d5-01d076fc547b",
        "url--3279c7de-8f91-5c1a-99d9-d6546c6c41f7",
        "email-message--6090e3d4-1fa8-5b36-9d2d-4a66d824995d",
        "relationship--a0fea125-c676-460d-bf7a-4099fdf6a976"
    ]
}


## D. Create the Alert Sighting

Tbis then represents the sighting of potential phishing email as a group of elements

### D.1 Create a Indicator with a Pattern for the Phishing Email
but set its status to `unkown` to start with

In [11]:
# 1. Setup pattern for indicator
pat1 = "[email-addr:value = '" + threat_email_addr["value"] + "' AND email:subject = '" + sus_msg["subject"] + "']"
# 2. Setup path to form and results
ind_path ="SDO/Indicator/indicator_alert.json"
results_path ="step1/indicator_alert.json"
# 3. Invoke the Make Observed Data Block
ind_1 = invoke_make_indicator_block(ind_path, results_path, pattern=pat1)
# 4. Add the record to the in-session bundles and lists
context_type = {
    "context_type": "unattached"
}
indicator_results_context_path = results_base + "step1/indicator_context.json"
result = invoke_save_incident_context_block(results_base + results_path, indicator_results_context_path, context_type)
print(f" result->{result}")

inputfile->../Block_Families/StixORM/SDO/Indicator/indicator_alert.json
path exists
input->{'indicator_form': {'base_required': {'type': 'identity', 'spec_version': '2.1', 'id': '', 'created': '', 'modified': ''}, 'base_optional': {'created_by_ref': '', 'revoked': None, 'labels': [], 'lang': '', 'external_references': [], 'object_marking_refs': [], 'granular_markings': []}, 'object': {'name': 'Potential Phishing Email', 'description': 'Suspicious email reported by user', 'pattern_type': 'stix', 'pattern': '', 'indicator_types': ['unknown']}, 'extensions': {}, 'sub': {}}, 'pattern': "[email-addr:value = 'evil@northkorea.nk' AND email:subject = 'we are coming for you']"}
pattern [email-addr:value = 'evil@northkorea.nk' AND email:subject = 'we are coming for you']
{
    "type": "indicator",
    "spec_version": "2.1",
    "id": "indicator--2d6468ca-e179-4b3b-9a00-3a40f3fa8d91",
    "created": "2024-08-22T07:37:59.470Z",
    "modified": "2024-08-22T07:37:59.470Z",
    "name": "Potential Phi

### D.2 Create the Sighting
Connecting the:
- observed-data object
- indicator object
- the reporter identity as the location

In [5]:
# 1. Setup path to form and results
sighting_data_path ="SRO/Sighting/sighting_alert.json"
results_path ="step1/sighting_alert.json"
# 2. Setup the SDO sighted, the Observed-Data that was observed with generated objects,
#                        then the identity object from the context storage (note the slight difference in indexing
sighted = ""
observation_list = []
where_list = []
# 2. Invoke the Make Observed Data Block
sight1 = invoke_sighting_block(sighting_data_path, results_path, observed=observation_list, sighted=sighted, where=where_list)
# 3. Add the record to the in-session bundles and lists
context_type = {
    "context_type": "unattached"
}
sighting_results_context_path = results_base + "step1/sighting_context.json"
# result = invoke_save_incident_context_block(results_base + results_path, sighting_results_context_path, context_type)
# print(f" result->{result}")

ExtraPropertiesError: Unexpected properties for Sighting: (observed_data_refs ).

### D.3 Promote Everything from Unattached to Other

Everything that uis connected in the Sighting should now be promoted, so move Unattached to the Other list

In [13]:
total_list = observed_list
total_list.append(obs_1)
total_list.append(ind_1)
total_list.append(sight1)
total_list.append(reporter_identity)

obs_context_move_path = results_base + "step1/context/context_move.json"
obs_context_move_results = results_base + "step1/context/context_move_results.json"
result = invoke_move_unattached_to_other_block(obs_context_move_path, obs_context_move_results, total_list)

delete element 0, node id email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf212
delete element 1, node id user-account--597ad4d4-35ba-585d-8f6d-134a75032f9b
delete element 0, node id email-addr--4722424c-7012-56b0-84d5-01d076fc547b
delete element 2, node id url--3279c7de-8f91-5c1a-99d9-d6546c6c41f7
delete element 2, node id email-message--6090e3d4-1fa8-5b36-9d2d-4a66d824995d
delete element 2, node id observed-data--3a124ba7-bba0-4c62-938b-03f3acb71097
delete element 2, node id indicator--2d6468ca-e179-4b3b-9a00-3a40f3fa8d91
delete element 0, node id identity--c8a60730-4fa0-4da0-90b2-5d1b548a35aa


## E. Create the Event Objects
Tbis Event represents the receiving of the phishing email

In [14]:
# 1. Setup path to form and results
event_data_path ="SDO/Event/event_alert.json"
results_path ="step1/event_alert.json"
# 2. Setup the SDO sighted, the Observed-Data that was observed with generated objects,
#                        then the identity object from the context storage (note the slight difference in indexing
sighting = sight1["id"]
# 2. Invoke the Make Observed Data Block
event_1 = invoke_make_event_block(event_data_path, results_path, changed_objects=None,sighting_refs=sighting)
# 3. Add the record to the in-session bundles and lists
context_type = {
    "context_type": "event"
}
event_results_context_path = results_base + "step1/context/event_context.json"
result = invoke_save_incident_context_block(results_base + results_path, event_results_context_path, context_type)
print(f" result->{result}")

{
    "type": "event",
    "spec_version": "2.1",
    "id": "event--fc4cab35-4fbb-494b-b78e-e72cb04badf0",
    "created": "2024-08-22T07:37:59.866Z",
    "modified": "2024-08-22T07:37:59.866Z",
    "description": "Suspicious email reported by user",
    "end_time": "2024-08-22T07:37:59.866Z",
    "event_types": [
        "dissemination-phishing-emails"
    ],
    "name": "Potential Phishing Email",
    "start_time": "2024-08-22T07:37:59.866Z",
    "extensions": {
        "extension-definition--4ca6de00-5b0d-45ef-a1dc-ea7279ea910e": {
            "extension_type": "new-sdo"
        }
    }
}
 result->{'context_result': ' incident context saved -> event\nstix_id -> event--fc4cab35-4fbb-494b-b78e-e72cb04badf0'}



### E.2 Then Create the Sequence Object for the Event

The Sequence object is created to place the event in a sequence, in this case a single-step

In [15]:
# 1. Setup path to form and results
sequence_data_path ="SDO/Sequence/sequence_alert.json"
results_path ="step1/sequence_event_single.json"
# 2. Setup the Sequence Object for the Event
#
step_type = "single_step"
sequence_type = "event"
sequenced_object = event_1["id"]
# 2. Invoke the Make Observed Data Block
seq_E_1 = invoke_make_sequence_block(sequence_data_path, results_path, step_type=step_type, sequence_type=sequence_type, sequenced_object=sequenced_object, on_completion=None, on_success=None, on_failure=None, next_steps=None)
# 3. Add the record to the in-session bundles and lists
context_type = {
    "context_type": "sequence"
}
sequence_results_context_path = results_base + "step1/event_sequence_context.json"
result = invoke_save_incident_context_block(results_base + results_path, sighting_results_context_path, context_type)
print(f" result->{result}")

{
    "type": "sequence",
    "spec_version": "2.1",
    "id": "sequence--2ebbaf5b-196b-4739-ab53-8546853c875c",
    "created": "2024-08-22T07:37:59.921Z",
    "modified": "2024-08-22T07:37:59.921Z",
    "sequenced_object": "event--fc4cab35-4fbb-494b-b78e-e72cb04badf0",
    "sequence_type": "event",
    "step_type": "single_step",
    "extensions": {
        "extension-definition--be0c7c79-1961-43db-afde-637066a87a64": {
            "extension_type": "new-sdo"
        }
    }
}
 result->{'context_result': ' incident context saved -> sequence\nstix_id -> sequence--2ebbaf5b-196b-4739-ab53-8546853c875c'}



### E.3 Since it is the First Sequence, Create the Start Sequence Object and Connect it

The Sequence object is created to place the event in a sequence, in this case a start-step

In [16]:
# 1. Setup path to form and results
sequence_data_path ="SDO/Sequence/sequence_start.json"
results_path ="step1/sequence_event_start.json"
# 2. Setup the Sequence Object for the Event
#
step_type = "start_step"
sequence_type = "event"
on_completion = seq_E_1["id"]
# 2. Invoke the Make Observed Data Block
seq_E_0 = invoke_make_sequence_block(sequence_data_path, results_path, step_type=step_type, sequence_type=sequence_type, sequenced_object=None, on_completion=on_completion, on_success=None, on_failure=None, next_steps=None)
# 3. Add the record to the in-session bundles and lists
context_type = {
    "context_type": "start"
}
event_results_context_path = results_base + "step1/event_seq_start_context.json"
result = invoke_save_incident_context_block(results_base + results_path, sighting_results_context_path, context_type)
print(f" result->{result}")

{
    "type": "sequence",
    "spec_version": "2.1",
    "id": "sequence--07bbdf40-f794-48f2-ae04-22b226c4fc9a",
    "created": "2024-08-22T07:37:59.985Z",
    "modified": "2024-08-22T07:37:59.985Z",
    "sequence_type": "event",
    "step_type": "start_step",
    "on_completion": "sequence--2ebbaf5b-196b-4739-ab53-8546853c875c",
    "extensions": {
        "extension-definition--be0c7c79-1961-43db-afde-637066a87a64": {
            "extension_type": "new-sdo"
        }
    }
}
 result->{'context_result': ' incident context saved -> start\nstix_id -> sequence--07bbdf40-f794-48f2-ae04-22b226c4fc9a'}


## F. Create the Task Objects

Tbis Task represents the first Task to clarify the situation first,no real Impactyet

In [17]:
# 1. Setup path to form and results
task_data_path ="SDO/Task/task_alert.json"
results_path ="step1/task_alert.json"
# 2. Invoke the Make Observed Data Block
task_1 = invoke_make_task_block(task_data_path, results_path, changed_objects=None)
# 3. Add the record to the in-session bundles and lists
context_type = {
    "context_type": "task"
}
task_results_context_path = results_base + "step1/task_context.json"
result = invoke_save_incident_context_block(results_base + results_path, task_results_context_path, context_type)
print(f" result->{result}")

{
    "type": "task",
    "spec_version": "2.1",
    "id": "task--0addefdc-2a6d-4f4e-a47f-859cda51ef3e",
    "created": "2024-08-22T07:38:00.780Z",
    "modified": "2024-08-22T07:38:00.780Z",
    "description": "Suspicious email reported by user",
    "end_time": "2024-08-22T07:38:00.780Z",
    "name": "Potential Phishing Email",
    "start_time": "2024-08-22T07:38:00.780Z",
    "extensions": {
        "extension-definition--2074a052-8be4-4932-849e-f5e7798e0030": {
            "extension_type": "new-sdo"
        }
    }
}
 result->{'context_result': ' incident context saved -> task\nstix_id -> task--0addefdc-2a6d-4f4e-a47f-859cda51ef3e'}


### F.2 Then Create the Sequence Object for the Task

The Sequence object is created to place the Task in a sequence, in this case a single-step

In [18]:
# 1. Setup path to form and results
sequence_data_path ="SDO/Sequence/sequence_alert.json"
results_path ="step1/sequence_task_single.json"
# 2. Setup the Sequence Object for the Event
#
step_type = "single_step"
sequence_type = "task"
sequenced_object = task_1["id"]
# 2. Invoke the Make Observed Data Block
seq_T_1 = invoke_make_sequence_block(sequence_data_path, results_path, step_type=step_type, sequence_type=sequence_type, sequenced_object=sequenced_object, on_completion=None, on_success=None, on_failure=None, next_steps=None)
# 3. Add the record to the in-session bundles and lists
context_type = {
    "context_type": "sequence"
}
event_sequence_results_context_path = results_base + "step1/task_sequence_context.json"
result = invoke_save_incident_context_block(results_base + results_path, event_sequence_results_context_path, context_type)
print(f" result->{result}")

{
    "type": "sequence",
    "spec_version": "2.1",
    "id": "sequence--743c4de2-4342-489c-bd7f-260a3b83b4e8",
    "created": "2024-08-22T07:38:00.139Z",
    "modified": "2024-08-22T07:38:00.139Z",
    "sequenced_object": "task--0addefdc-2a6d-4f4e-a47f-859cda51ef3e",
    "sequence_type": "task",
    "step_type": "single_step",
    "extensions": {
        "extension-definition--be0c7c79-1961-43db-afde-637066a87a64": {
            "extension_type": "new-sdo"
        }
    }
}
 result->{'context_result': ' incident context saved -> sequence\nstix_id -> sequence--743c4de2-4342-489c-bd7f-260a3b83b4e8'}



### F.3 Then Create the Sequence Start Object for the Tasks

The Sequence object is created to place the Task in a sequence, in this case a start-step

In [19]:
# 1. Setup path to form and results
sequence_data_path ="SDO/Sequence/sequence_start.json"
results_path ="step1/sequence_task_start.json"
# 2. Setup the Sequence Object for the Event
#
step_type = "start_step"
sequence_type = "task"
on_completion = seq_T_1["id"]
# 2. Invoke the Make Observed Data Block
seq_T_0 = invoke_make_sequence_block(sequence_data_path, results_path, step_type=step_type, sequence_type=sequence_type, sequenced_object=None, on_completion=on_completion, on_success=None, on_failure=None, next_steps=None)
# 3. Add the record to the in-session bundles and lists
context_type = {
    "context_type": "start"
}
task_seq_results_context_path = results_base + "step1/task_seq_start_context.json"
result = invoke_save_incident_context_block(results_base + results_path, task_seq_results_context_path, context_type)
print(f" result->{result}")

{
    "type": "sequence",
    "spec_version": "2.1",
    "id": "sequence--71207b40-61fd-4d35-a106-7ea07c62f595",
    "created": "2024-08-22T07:38:00.202Z",
    "modified": "2024-08-22T07:38:00.202Z",
    "sequence_type": "task",
    "step_type": "start_step",
    "on_completion": "sequence--743c4de2-4342-489c-bd7f-260a3b83b4e8",
    "extensions": {
        "extension-definition--be0c7c79-1961-43db-afde-637066a87a64": {
            "extension_type": "new-sdo"
        }
    }
}
 result->{'context_result': ' incident context saved -> start\nstix_id -> sequence--71207b40-61fd-4d35-a106-7ea07c62f595'}


## H. Write out the Contex Memory for the Incident

Export out the Context Memory for the Incident

In [20]:
# Save the Tpe Refinery Context Memory File